In [4]:
import pandas as pd
import numpy as np

In [2]:
#Artur
df = pd.read_excel('C:/Users/ScMa9004/Downloads/final DB final Factor_W2.xlsx', sheet_name = 'non LSC', low_mempry = False)

In [3]:
#df.loc[df['type_pnl'].isna(), 'type_pnl'] = df['Otype Code']

In [4]:
arc = pd.read_excel('C:/Users/ScMa9004/Documents/All Russia cities 01.01.2018.xlsx', sheet_name = 'All_Russia_cities', skiprows = 1)

In [5]:
arc = arc[['City Code', 'region', 'pgr']]

In [6]:
arc = arc.rename(columns = {'City Code': 'ccode_'})

# UNIVERSE COCA PEPSI

In [7]:
wrk = df[(df['type_pnl'].isin(['OKS','OLA','OME','OMM', 'OMS', 'OPV', 'OSM', 'OSU', 'OWS'])) &
         (df['coop'].isin([0]))]

In [136]:
wrk = pd.merge(wrk, arc, how = 'left')

In [137]:
wrk.loc[(wrk['ccode_'].isin([1, 2])) &
        (wrk['type_pnl'].isin(['OKS', 'OPV'])), 'm1'] = wrk['ccode_'].astype('str') + '-Impulse stores'
wrk.loc[(wrk['ccode_'].isin([1, 2])) &
        (wrk['type_pnl'].isin(['OSU'])), 'm1'] = wrk['ccode_'].astype('str') + '-Supermarkets'
wrk.loc[(wrk['ccode_'].isin([1, 2])) &
        (wrk['type_pnl'].isin(['OLA', 'OME', 'OMM', 'OMS'])) &
        (wrk['TTR'] == 'MT'), 'm1'] = wrk['ccode_'].astype('str') + '-Supperettes'
wrk.loc[(wrk['ccode_'].isin([1, 2])) &
        (wrk['type_pnl'].isin(['OLA', 'OME', 'OMM', 'OMS', 'OSM', 'OWS'])) &
        (wrk['TTR'] == 'TT'), 'm1'] = wrk['ccode_'].astype('str') + '-Traditional Food'

In [138]:
wrk.loc[(wrk['ccode_'] > 2) &
        (wrk['ccode_'] < 25), 'm1'] = wrk['ccode_'].astype('str') + '-' + wrk['TTR'].astype('str')

In [139]:
wrk.loc[wrk['m1'].isna(), 'm1'] = wrk['region'] + '-' + wrk['pgr'].astype('int').astype('str') + '-' + wrk['TTR']

In [140]:
wrk.loc[(((wrk['coca'] == 0) & (wrk['pepsi'] == 0)) | ((wrk['coca'] == 1) & (wrk['pepsi'] == 1))), 'shmoka'] = 'blank'
wrk.loc[(wrk['coca'] == 1) & (wrk['pepsi'] == 0), 'shmoka'] = 'coca'
wrk.loc[(wrk['coca'] == 0) & (wrk['pepsi'] == 1), 'shmoka'] = 'pepsi'

In [141]:
wrk['coca'] = np.nan
wrk['pepsi'] = np.nan
wrk['blank'] = np.nan

In [142]:
wrk.loc[wrk['shmoka'] == 'coca', 'coca'] = wrk['NUHTRP Final']
wrk.loc[wrk['shmoka'] == 'pepsi', 'pepsi'] = wrk['NUHTRP Final']
wrk.loc[wrk['shmoka'] == 'blank', 'blank'] = wrk['NUHTRP Final']

In [143]:
universe = pd.pivot_table(wrk, index = 'm1', values = ['coca', 'pepsi', 'blank', 'NUHTRP Final'], aggfunc = np.sum)

In [144]:
universe['coca_universe'] = universe['coca'] / universe['NUHTRP Final']
universe['pepsi_universe'] = universe['pepsi'] / universe['NUHTRP Final']
universe['blank_universe'] = universe['blank'] / universe['NUHTRP Final']

In [145]:
universe = universe.reset_index(drop = False)
universe = universe[['m1', 'coca_universe', 'pepsi_universe', 'blank_universe']]

In [146]:
universe.to_excel('universe_coca_pepsi.xlsx', index = False)

# UNIVERSE BABY

In [147]:
wrk = df[(df['type_pnl'].isin(['OBS'])) & (df['coop'].isin([0]))]

In [148]:
wrk = pd.merge(wrk, arc, how = 'left')

In [149]:
wrk.loc[wrk['ccode_'] < 25, 'merge'] = wrk['ccode_'].astype('str')

In [150]:
wrk.loc[wrk['ccode_'] > 24, 'merge'] = wrk['region'] + '-' + wrk['pgr'].astype('int').astype('str')

In [151]:
wrk.loc[wrk['hl141'] == 1, 'hl141'] = wrk['NUHTRP Final']
wrk.loc[wrk['hl088'] == 1, 'hl088'] = wrk['NUHTRP Final']

In [154]:
universe = pd.pivot_table(wrk, index = 'merge', values = ['hl141', 'hl088', 'NUHTRP Final'], aggfunc = np.sum).reset_index()

In [155]:
universe['universe_baby_f'] = universe['hl141'] / universe['NUHTRP Final']
universe['universe_baby_d'] = universe['hl088'] / universe['NUHTRP Final']

In [156]:
universe = universe[['merge', 'universe_baby_f', 'universe_baby_d']]

In [274]:
universe.to_excel('universe_baby.xlsx', index = False)

# CHA CHECK

In [32]:
df['key'] = df['ccode_'].astype('str') + '-' + df['type_pnl'].astype('str') + '-' + df['TTR'].astype('str')

In [33]:
wrk = df[['key', 'cha', 'NUHTRP Final']][df['coop'].isin([0])]

In [34]:
fmcg = pd.read_excel('C:/Users/ScMa9004/Documents/RRES/FMCG_Urban_RES_db_Community_File_cut.xlsx')
cig = pd.read_excel('C:/Users/ScMa9004/Documents/RRES/CIG_Zuniverse allocation_Community_File.xlsx', sheet_name = 'CIG_Urban_RES_db')
beer = pd.read_excel('C:/Users/ScMa9004/Documents/RRES/BEER_Urban_RES_db_Community_File.xlsx', sheet_name = 'BEER_Urban_RES_db')

In [35]:
fmcg['key'] = fmcg['CCODE'].astype('str') + '-' + fmcg['RMS_TYPE'].astype('str') + '-' + fmcg['TTR'].astype('str')
cha_check_fmcg = fmcg[['key', 'Final Cell ID', 'CELL_NAME']]
cha_check_fmcg = cha_check_fmcg[cha_check_fmcg['Final Cell ID'] != 0]
cha_check_fmcg = cha_check_fmcg.drop_duplicates()
cha_check_fmcg = pd.merge(cha_check_fmcg, wrk, how = 'right', on = 'key')
cha_check_fmcg.loc[cha_check_fmcg['cha'].isna(), 'cha'] = 'Standalone'
cha_check_fmcg = pd.pivot_table(cha_check_fmcg, index = ['Final Cell ID', 'key', 'cha', 'CELL_NAME'],
                                values = 'NUHTRP Final',
                                aggfunc = np.sum).reset_index(drop = False)

In [36]:
cig['key'] = cig['CCODE'].astype('str') + '-' + cig['RMS_TYPE'].astype('str') + '-' + cig['TTR'].astype('str')
cha_check_cig = cig[['key', 'Final cell id', 'CELL_NAME']]
cha_check_cig = cha_check_cig.rename(columns = {'Final cell id': 'Final Cell ID'})
cha_check_cig = cha_check_cig[cha_check_cig['Final Cell ID'] != 0]
cha_check_cig = cha_check_cig.drop_duplicates()
cha_check_cig = pd.merge(cha_check_cig, wrk, how = 'right', on = 'key')
cha_check_cig.loc[cha_check_cig['cha'].isna(), 'cha'] = 'Standalone'
cha_check_cig = pd.pivot_table(cha_check_cig, index = ['Final Cell ID', 'key', 'cha', 'CELL_NAME'],
                                values = 'NUHTRP Final',
                                aggfunc = np.sum).reset_index(drop = False)

In [37]:
beer['key'] = beer['CCODE'].astype('str') + '-' + beer['RMS_TYPE'].astype('str') + '-' + beer['TTR'].astype('str')
cha_check_beer = beer[['key', 'CELLID', 'CELL_NAME']]
cha_check_beer = cha_check_beer.rename(columns = {'CELLID': 'Final Cell ID'})
cha_check_beer = cha_check_beer[cha_check_beer['Final Cell ID'] != 0]
cha_check_beer = cha_check_beer.drop_duplicates()
cha_check_beer = pd.merge(cha_check_beer, wrk, how = 'right', on = 'key')
cha_check_beer.loc[cha_check_beer['cha'].isna(), 'cha'] = 'Standalone'
cha_check_beer = pd.pivot_table(cha_check_beer, index = ['Final Cell ID', 'key', 'cha', 'CELL_NAME'],
                                values = 'NUHTRP Final',
                                aggfunc = np.sum).reset_index(drop = False)

In [38]:
with pd.ExcelWriter('cha_check_universe.xlsx') as writer:
    cha_check_fmcg.to_excel(writer, sheet_name = 'fmcg')
    cha_check_cig.to_excel(writer, sheet_name = 'cig')
    cha_check_beer.to_excel(writer, sheet_name = 'beer')

# AP_OP

In [2]:
path = 'C:/Users/ScMa9004/Documents/Итоги визита/313/'

In [5]:
pr = pd.read_excel(path + 'Panel Russia_for 313.xlsx', sheet_name = 'Panel', skiprows = 2)

In [6]:
AP_OP_fmcg = pd.pivot_table(pr, index = 'MAS', values = ['stand', 'act stand'], aggfunc = np.sum).reset_index()
AP_OP_fmcg = AP_OP_fmcg.rename(columns = {'MAS': 'cell', 'stand': 'OP_fmcg', 'act stand': 'AP_fmcg'})

In [7]:
AP_OP_cig = pd.pivot_table(pr, index = 'MAS.1', values = ['cig', 'act cig '], aggfunc = np.sum).reset_index()
AP_OP_cig = AP_OP_cig.rename(columns = {'MAS.1': 'cell', 'cig': 'OP_cig', 'act cig ': 'AP_cig'})

In [8]:
AP_OP_beer = pd.pivot_table(pr, index = 'MAS.2', values = ['beer ', 'act beer '], aggfunc = np.sum).reset_index()
AP_OP_beer = AP_OP_beer.rename(columns = {'MAS.2': 'cell', 'beer ': 'OP_beer', 'act beer ': 'AP_beer'})

In [9]:
AP_OP = AP_OP_fmcg.append(AP_OP_cig, sort = False).append(AP_OP_beer, sort = False)

In [10]:
AP_OP = AP_OP.reset_index(drop = True)

In [11]:
AP_OP.to_excel('OP.xlsx')